https://stackoverflow.com/questions/45801494/converting-excel-solver-solution-to-python-pulp

In [1]:
%load_ext nb_black

import pandas as pd

from pulp import LpProblem, LpStatus, LpVariable, LpAffineExpression
from pulp import LpBinary, LpMinimize, LpMaximize, lpSum, value

<IPython.core.display.Javascript object>

In [2]:
df = pd.DataFrame(
    [
        ["Receiving", 61, 7.75, 11_346],
        ["Picking", 94, 7.75, 13_011],
        ["PPicking", 32, 7.75, 2_715],
        ["QC", 63, 7.75, 13_682],
        ["Packing", 116, 7.75, 14_194],
    ],
    columns=["Dept", "Target", "Hours/Day", "Production"],
)

df.head()

,Dept,Target,Hours/Day,Production
0,Receiving,61,7.75,11346
1,Picking,94,7.75,13011
2,PPicking,32,7.75,2715
3,QC,63,7.75,13682
4,Packing,116,7.75,14194


<IPython.core.display.Javascript object>

In [3]:
HOUR_COUNT = LpVariable.dicts("HOUR_COUNT", df.index, lowBound=0, cat="Continuous")
OVERTIME = LpVariable.dicts("OVERTIME", df.index, lowBound=0, cat="Continuous")

model = LpProblem("OT Reduction", LpMinimize)

/usr/local/lib/python3.7/site-packages/pulp/pulp.py:1137: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


<IPython.core.display.Javascript object>

In [4]:
# Objective function
model += lpSum([OVERTIME[idx] for idx in df.index])

# Lower and Upper bounds:
for index, row in df.iterrows():
    model += (
        row.Target * row["Hours/Day"] * HOUR_COUNT[index] + row.Target * OVERTIME[index]
        >= row.Production
    )

# Total HOUR_COUNT should be less than or equal to 92
model += lpSum([HOUR_COUNT[idx] for idx in df.index]) <= 92

<IPython.core.display.Javascript object>

In [5]:
# Solve model
model.solve()

LpStatus[model.status]

'Optimal'

<IPython.core.display.Javascript object>

In [6]:
output = df[["Dept"]].copy()
output["HC"] = [HOUR_COUNT[i].varValue for i in df.index]
output["OT"] = [OVERTIME[i].varValue for i in df.index]

output.round(2)

,Dept,HC,OT
0,Receiving,24.00,0.0
1,Picking,13.24,35.8
2,PPicking,10.95,0.0
3,QC,28.02,0.0
4,Packing,15.79,0.0


<IPython.core.display.Javascript object>